In [22]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer, TrainerCallback
from peft import LoraConfig, TaskType, get_peft_model
import torch
import psutil
import numpy as np



In [23]:
torch.cuda.set_per_process_memory_fraction(1.0, 0)  # Use maximum available memory
torch.cuda.memory_max_split_size_mb = 64  # Set the max split size to avoid fragmentation

In [24]:
def print_memory_footprint():
    # GPU memory usage
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.memory_allocated() / (1024 ** 3)  # Convert to GB
        gpu_memory_cached = torch.cuda.memory_reserved() / (1024 ** 3)  # Cached memory
        print(f"[GPU] Memory Allocated: {gpu_memory:.2f} GB, Cached: {gpu_memory_cached:.2f} GB")
    else:
        print("[GPU] No GPU detected.")

    # CPU memory usage
    memory = psutil.virtual_memory()
    used_memory_gb = memory.used / (1024 ** 3)  # Convert to GB
    total_memory_gb = memory.total / (1024 ** 3)
    print(f"[CPU] Memory Usage: {used_memory_gb:.2f} GB / {total_memory_gb:.2f} GB")

In [25]:
'''
print("First example of blended_skill_talk training set:")
print(dataset['train'][0])
'''

'\nprint("First example of blended_skill_talk training set:")\nprint(dataset[\'train\'][0])\n'

In [26]:
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have a pad token, so we use eos_token

In [27]:
# # Load dataset and tokenizer
# dataset2 = load_dataset("allenai/soda")



# def tokenize_function(examples):
#     # Concatenate dialog turns into a single string for language modeling
#     texts = [" ".join(dialog) for dialog in examples["dialogue"]]
#     return tokenizer(texts, truncation=True, max_length=512)

# # Tokenize datasets
# tokenized_datasets = dataset2.map(tokenize_function, batched=True, remove_columns=['head', 'relation', 'tail', 'literal', 'narrative', 'dialogue', 'speakers', 'PersonX', 'PersonY', 'PersonZ', 'original_index', 'split', 'head_answer', 'pmi_head_answer', 'relation_tail_answer', 'pmi_relation_tail_answer'])
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(2000))
# small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(500))

In [28]:
print(dataset2)
tokenized_datasets


DatasetDict({
    train: Dataset({
        features: ['head', 'relation', 'tail', 'literal', 'narrative', 'dialogue', 'speakers', 'PersonX', 'PersonY', 'PersonZ', 'original_index', 'split', 'head_answer', 'pmi_head_answer', 'relation_tail_answer', 'pmi_relation_tail_answer'],
        num_rows: 1191582
    })
    validation: Dataset({
        features: ['head', 'relation', 'tail', 'literal', 'narrative', 'dialogue', 'speakers', 'PersonX', 'PersonY', 'PersonZ', 'original_index', 'split', 'head_answer', 'pmi_head_answer', 'relation_tail_answer', 'pmi_relation_tail_answer'],
        num_rows: 146346
    })
    test: Dataset({
        features: ['head', 'relation', 'tail', 'literal', 'narrative', 'dialogue', 'speakers', 'PersonX', 'PersonY', 'PersonZ', 'original_index', 'split', 'head_answer', 'pmi_head_answer', 'relation_tail_answer', 'pmi_relation_tail_answer'],
        num_rows: 148968
    })
})


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1191582
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 146346
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 148968
    })
})

In [29]:
# LoRA configuration for causal language modeling
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=4,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj"],
)

# %%

In [30]:
# Dynamic device assignment
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained GPT-2 language model
# Load pre-trained GPT-2 model
#model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(device)
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0",
                                             torch_dtype=torch.bfloat16,).to(device)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Enable gradient checkpointing if you run into memory issues
#model.gradient_checkpointing_enable()

# %%
# Print the model's architecture to inspect the names of the modules
#print(model)

# %%
print_memory_footprint()

Using device: cuda
[GPU] Memory Allocated: 10.27 GB, Cached: 11.01 GB
[CPU] Memory Usage: 3.88 GB / 31.35 GB


In [31]:
# %%

# Data collator for language modeling (masks tokens for prediction)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) # mlm=False for causal LM

# Perplexity as metric
#perplexity_metric = evaluate.load("perplexity", module_type="metric")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    shift_logits = torch.tensor(logits[:, :-1, :])
    shift_labels = torch.tensor(labels[:, 1:])
    loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    ppl = torch.exp(loss).item()
    return {"perplexity": ppl}

# Training args
training_args = TrainingArguments(
    output_dir="./lama-dialog-finetuned",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_dir="./logs",
    logging_steps=1000,
    #evaluation_strategy="epoch",
    #save_strategy="epoch",
    report_to="none",
    remove_unused_columns=False,
    bf16=True,  # Enable bfloat16
    fp16=False,  # Disable fp16 to avoid conflicts
)
# %%
print_memory_footprint()

[GPU] Memory Allocated: 10.27 GB, Cached: 11.01 GB
[CPU] Memory Usage: 3.89 GB / 31.35 GB


In [32]:
class MemoryCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        print("\nMemory footprint after evaluation:")
        print_memory_footprint()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    #eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
    callbacks=[MemoryCallback()]
)


/tmp/ipykernel_35/3187942852.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [33]:

print("--- TinyLlama Before Fine-tuning ---")
model_before_finetune = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0",
             torch_dtype=torch.bfloat16).to(device)

input_text_before = "Hello, how are you?"
input_ids_before = tokenizer.encode(input_text_before, return_tensors="pt").to(device)

print(f"Input: {input_text_before}")
output_before = model_before_finetune.generate(
    input_ids_before,
    max_length=50,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)
generated_response_before = tokenizer.decode(output_before[0], skip_special_tokens=True)
print(f"Generated Response (Before): {generated_response_before}")
print("---")


--- TinyLlama Before Fine-tuning ---
Input: Hello, how are you?


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Generated Response (Before): Hello, how are you?
I am fine, thank you.
How are you? I am fine, thank you.
Can you repeat that again?
Can you repeat that again, please?
Can you repeat that again,
---


In [34]:

print("--- TinyLlama Before Fine-tuning ---")
#model_before_finetune = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

input_text_before = "The USA celebrate independence day on "
input_ids_before = tokenizer.encode(input_text_before, return_tensors="pt").to(device)

print(f"Input: {input_text_before}")
output_before = model_before_finetune.generate(
    input_ids_before,
    max_length=50,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)
generated_response_before = tokenizer.decode(output_before[0], skip_special_tokens=True)
print(f"Generated Response (Before): {generated_response_before}")
print("---")


--- TinyLlama Before Fine-tuning ---
Input: The USA celebrate independence day on 
Generated Response (Before): The USA celebrate independence day on 4th July.

2. The USA celebrate independence day on 4th July.

3. The USA celebrate independence day on 4th July.

4. The
---


In [35]:
any(p.requires_grad for p in model.parameters())

True

In [36]:
trainer.train()

Step,Training Loss
1000,0.737500


TrainOutput(global_step=1500, training_loss=0.7328169759114583, metrics={'train_runtime': 1826.0045, 'train_samples_per_second': 3.286, 'train_steps_per_second': 0.821, 'total_flos': 9440523922636800.0, 'train_loss': 0.7328169759114583, 'epoch': 3.0})

In [37]:
trainer.save_model('TinyLlama-new-1000')

In [38]:
print_memory_footprint()

[GPU] Memory Allocated: 8.22 GB, Cached: 13.81 GB
[CPU] Memory Usage: 3.88 GB / 31.35 GB


In [39]:
# Dynamic device assignment
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# For generation, we'll load the model and use the `generate` method
model_for_generation = AutoModelForCausalLM.from_pretrained('TinyLlama-new-1000').to(device)
#model_for_generation = get_peft_model(model_for_generation, lora_config) # Ensure LoRA is applied

input_text = "Hello, how are you?"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

print("\nGenerating response:")
output = model_for_generation.generate(
    input_ids,
    max_length=150,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Input: {input_text}")
print(f"Generated Response: {generated_response}")

print_memory_footprint()

Using device: cuda

Generating response:


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Input: Hello, how are you?
Generated Response: Hello, how are you? I'm doing well. I'm just getting started with my new job. That sounds great! I'm excited to start working and see how it goes. I'm sure it will be great. I'm sure it will be. I'm looking forward to it too. I'm excited to see what the future holds for me. I'm excited too. I'm looking forward to seeing how my new job will impact my life. I'm looking forward to seeing how it will impact my life too. I'm excited to see how my new job will impact my life. I'm excited to see how it will impact my life too. I'm excited
[GPU] Memory Allocated: 8.22 GB, Cached: 12.75 GB
[CPU] Memory Usage: 4.51 GB / 31.35 GB


In [40]:
input_text = "The USA celebrate independence day on "
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

print("\nGenerating response:")
output = model_for_generation.generate(
    input_ids,
    max_length=150,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Input: {input_text}")
print(f"Generated Response: {generated_response}")


Generating response:
Input: The USA celebrate independence day on 
Generated Response: The USA celebrate independence day on 4th July.

2. India Independence Day is celebrated on 15th August.

3. Canada celebrates its independence day on 1st October.

4. Australia celebrates its independence day on 26th June.

5. New Zealand celebrates its independence day on 5th October.

These are just a few examples, but there are many more countries that celebrate independence day on different dates.


In [41]:
input_text = "Explain how gravity works?. Can you?"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

print("\nGenerating response:")
output = model_for_generation.generate(
    input_ids,
    max_length=250,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Input: {input_text}")
print(f"Generated Response: {generated_response}")


Generating response:
Input: Explain how gravity works?. Can you?
Generated Response: Explain how gravity works?. Can you?
Gravity is the force that holds the Earth and all other objects in the universe together. It is a force that is present in all objects in the universe, regardless of their size or shape. Gravity is a result of the attraction between two objects that are moving towards each other at a constant speed. The force of gravity is proportional to the masses of the two objects and inversely proportional to the square of the distance between them.
Gravity is a fundamental force of nature that is present in all objects in the universe. It is a result of the attraction between two objects that are moving towards each other at a constant speed. The force of gravity is proportional to the masses of the two objects and inversely proportional to the square of the distance between them.
Gravity is a force that is present in all objects in the universe, regardless of their size or s